In [1]:
!pip install -U git+https://github.com/huggingface/diffusers.git


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/diffusers.git to c:\users\platinum\appdata\local\temp\pip-req-build-j07wswc9
  Resolved https://github.com/huggingface/diffusers.git to commit d78acdedc1d248406455c6abcfa4224cd59f9ae7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 682.7 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 253.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? 

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git 'C:\Users\Platinum\AppData\Local\Temp\pip-req-build-j07wswc9'


In [2]:
!pip install -U controlnet_aux==0.0.7 


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/202.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/202.4 kB ? eta -:--:--
     ----- ------------------------------- 30.7/202.4 kB 435.7 kB/s eta 0:00:01
     ------- ----------------------------- 41.0/202.4 kB 326.8 kB/s eta 0:00:01
     ---------------- -------------------- 92.2/202.4 kB 521.8 kB/s eta 0:00:01
     --------------------- -------------- 122.9/202.4 kB 599.1 kB/s eta 0:00:01
     ------------------------------------ 202.4/202.4 kB 768.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/44.6 kB ? eta -:--:--
   ---------------------------------------- 44.6/44.6 kB ? eta 

In [3]:
!pip install transformers accelerate safetensors mediapipe

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     -------- ---------------------------- 30.7/134.8 kB 330.3 kB/s eta 0:00:01
     ---------------- -------------------- 61.4/134.8 kB 409.6 kB/s eta 0:00:01
     ------------------------------ ----- 112.6/134.8 kB 598.8 kB/s eta 0:00:01
     ------------------------------------ 134.8/134.8 kB 613.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/162.6 kB ? eta -:--:--
     -------------------------- ----------- 112.6/162.6 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 162.6/162.6 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.8 MB 1.9 MB/s eta 0:00:05
   - -------------------------------------- 0.2/8.8 MB 2.8 MB/s eta 0:00:04
   - -

In [4]:
import torch
from diffusers import StableDiffusionXLAdapterPipeline, T2IAdapter, EulerAncestralDiscreteScheduler, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from controlnet_aux.pidi import PidiNetDetector

C:\Users\Platinum\anaconda3\envs\Cuda-erf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Platinum\anaconda3\envs\Cuda-erf\lib\site-packages\controlnet_aux\segment_anything\modeling\tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
C:\Users\Platinum\anaconda3\envs\Cuda-erf\lib\site-packages\controlnet_aux\segment_anything\modeling\tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please 

In [6]:
# load adapter
adapter = T2IAdapter.from_pretrained(
  "TencentARC/t2i-adapter-sketch-sdxl-1.0", torch_dtype=torch.float, varient="fp16"
).to("cuda")

In [7]:

# load euler_a scheduler
model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
euler_a = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
vae=AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float)
pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    model_id, vae=vae, adapter=adapter, scheduler=euler_a, torch_dtype=torch.float, variant="fp16", 
).to("cuda")
#pipe.enable_xformers_memory_efficient_attention()

pidinet = PidiNetDetector.from_pretrained("lllyasviel/Annotators").to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:11<00:00,  1.66s/it]


In [10]:
url = "https://huggingface.co/Adapter/t2iadapter/resolve/main/figs_SDXLV1.0/org_sketch.png"
image = load_image(url)
image = pidinet(
  image, detect_resolution=1024, image_resolution=1024, apply_filter=True
)
prompt = "a robot, mount fuji in the background, 4k photo, highly detailed"
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image,
    num_inference_steps=30,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_sketch.png')

100%|██████████| 30/30 [48:42<00:00, 97.41s/it] 


In [13]:
image2 = load_image("tree.jpg")
image2 = pidinet(
  image2, detect_resolution=1024, image_resolution=1024, apply_filter=True
)
prompt = "a tree, mount in the background, 4k photo, highly detailed"
negative_prompt = ""

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image2,
    num_inference_steps=10,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_tree.png')

100%|██████████| 10/10 [14:51<00:00, 89.19s/it]


In [17]:
image2 = load_image("manga.jpeg")
image2 = pidinet(
  image2, detect_resolution=1024, image_resolution=1024, apply_filter=True
)
prompt = "a manga, purple jacket, blonde, green eyes, 4k photo, highly detailed"
negative_prompt = ""

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image2,
    num_inference_steps=10,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_manga.png')

100%|██████████| 10/10 [12:41<00:00, 76.14s/it]
